## Introduction to Finite Element Methods

<br>

### Gauss quadrature rules

<br>

### Yerlan Amanbek

<b> yerlan@utexas.edu </b>


<table style="border-style: hidden;">
<tr>
<td style="border-left: none"><img src="images/ices-logo-2017.png" width="350"></td>
</tr>
</table>


# Objective

 * Revisit numerical intergration methods
 * Gauss integration(optimal)
 


## Set-up

The notebook relies on Python code. To initialize the notebook, select **Cell->Run All Below**
 
<!-- * The efficacy of this approach will be judged by the
considering the size of the class of polynomials for which
this approximation formula gives exact results.
 * With 2n values to be determined, we should expect the
upper limit of the precision of this method to be 2n-1.
 * In other words, this method should be exact for
polynomials of degree 2n - 1 or less. -->
 

## Introduction

In general, case a mapping between elements and reference(master) elements is used to simplify the integration domain.

We need to compute the definite integral a lot in the FEM computations.

$$    I :=  \int_{a}^b f(x)\,dx$$

The integral can be viewed graphically as the area between the x-axis and the curve y = f (x) in the region of the limits of integration. Thus, we can interpret numerical integration as an approximation of that area.



<!--- The Newton formulas discussed so far have used equally spaced nodes in the interval $[a, b]$ of the form $x_j = a+jh$  for  $j \in \mathbb{Z}$,
where 
$$ h=\frac{b-a}{n} $$ 
for given $n \in \mathbb{N}$ -->


In [1]:
from ipywidgets import interact, fixed, IntSlider, FloatSlider, Text, Dropdown
%matplotlib inline
import areaTools as AT
import ipywidgets as widgets

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
## Do not change the text in this box
f_box = Text(value="exp(-x**2)", description=r'$f(x)=$')
n_slider = widgets.IntSlider(min=2,max=20, step=2, value=4, description=r'$n_{start}$')
method_type = Dropdown(options=['left', 'right', 'midpoint','trapezoid','simpson'],
                        value='left', description='Method:')
interact(AT.plot3Areas,f=f_box,a="-1.0",b="1.0",n=n_slider,method=method_type);

interactive(children=(Text(value='exp(-x**2)', description='$f(x)=$'), Text(value='-1.0', description='a'), Text(value='1.0', description='b'), IntSlider(value=4, description='$n_{start}$', max=20, min=2, step=2), Dropdown(description='Method:', options=('left', 'right', 'midpoint', 'trapezoid', 'simpson'), value='left'), Output()), _dom_classes=('widget-interact',))

Some quick explanations of how the notebook works:

**The meaning of the variables are as follows:**
- "a": the left endpoint of the integration interval
- "b": the right endpoint of the integration interval
- "n" the number of sub-intervals to use for approximating the integral
- "f(x)": the function to integrate.

**How to define the function:**
- x\*\*p means $x^p$. For example, x\*\*0.5 means $\sqrt x$
- for a constant function $f(x) = c$ use $c*(x**0)$ (every function definition needs an x in it!)
- for $e^p$ use exp(p) (so for $e$ use exp(1))
- for $\pi$ use pi
- type trig functions the way you would on your calculator(sin(x), cos(x), etc)

**Some functions you might want to try:**
- 4-x\*\*2
- cos(x)
- sin(x\*\*2)
- exp(x\*\*2)
- exp(-x\*\*2/2)/sqrt(2.\*pi) (this is the famous "normal curve")
- exp(x)/x (make sure that 0 is not part of your interval!)


$$     \int_{a}^b f(x)\,dx \approx \sum_{i=1}^n w_i f(x_i)$$

The trapezoidal rule of numerical integration simply
approximates the area by the sum of several equally spaced trapezoids under the curve between the limits of a and b. 

<table style="border-style: hidden;">
<tr>
<td style="border-left: none"><img src="images/integration_trapezoidal.png" width="250"></td>
</tr>
</table>

The height of a trapezoid is found from the integrand, $y_j = y ( x_j )$, evaluated at equally spaced points, $x_j$ and $x_{j+1}$. Thus, a typical contribution is $Area = h\frac{ y_j + y_{j+1}}{2}$, where $h = x_{j+1} − x_j $ is the spacing. Thus, for $n$ points (and
$n − 1$ spaces), the well-known approximation is
$$ I \approx h \left( \frac{1}{2} y_1+y_2+y_3+ ...+y_{n-1}+ \frac{1}{2} y_n \right)$$
where $w_j = h$, except $w_1 = w_n = \frac{h}{2}$.




In [4]:
## Do not change the text in this box
# -3*x**2+2*x+6
f_box = Text(value="exp(-x**2)", description=r'$f(x)=$')
n_slider = widgets.IntSlider(min=1,max=20, step=2, value=2, description=r'$n_{start}$')
method_type = Dropdown(options=['trapezoid'],
                        value='trapezoid', description='Method:')
interact(AT.plot3Areas,f=f_box,a="-1.0",b="1.0",n=n_slider,method=method_type);

interactive(children=(Text(value='exp(-x**2)', description='$f(x)=$'), Text(value='-1.0', description='a'), Text(value='1.0', description='b'), IntSlider(value=2, description='$n_{start}$', max=20, min=1, step=2), Dropdown(description='Method:', options=('trapezoid',), value='trapezoid'), Output()), _dom_classes=('widget-interact',))

# Gauss integration

The famous mathematician Gauss posed this question: 

* What is the minimum number of points, $n$, required to exactly integrate a polynomial?

* What are the corresponding function values and weights?

<table style="border-style: hidden;">
<tr>
<td style="border-left: none"><img src="images/trapezoidalrule.png" width="250"></td>
<td style="border-left: none"><img src="images/gaussian_quadrature.png" width="250"></td>  
    
</tr>
</table>


# Example: 

Let $[a, b] = [-1, 1]$ and $n=2$

We want to choose $x_1, x_2, w_1$ and $w_2$ so that

$$     \int_{-1}^1 f(x)\,dx \approx w_1 f(x_1) + w_2 f(x_2)$$

The approximation should be exact for any polynomial of degree 3 or less.

Let  $f(x) = a_0 + a_1x+a_2x^2+a_3x^3$

then

$\int_{-1}^1 f(x)\,dx = a_0\int_{-1}^1 1\,dx+a_1\int_{-1}^1 x\,dx+a_2\int_{-1}^1 x^2\,dx+a_3\int_{-1}^1 x^3\,dx$

Each of the definite integrals on the right-hand side has an
integrand of degree 3 or less. Gaussian Quadrature should be
exact for each of these.


Applying Gaussian Quadrature to each remaining integral
yields:

 $$\int_{-1}^1 1\,dx = 2 = w_1 + w_2$$
 
 $$\int_{-1}^1 x \,dx = 0 = w_1 x_1 +w_2 x_2$$
 
  
 $$\int_{-1}^1 x^2\,dx = \frac{2}{3} = w_1 x_1^2+w_2x_2^2$$
 
 $$ \int_{-1}^1 x^3\,dx = 0 =w_1x_1^3 +w_2 x_2^3$$

 $w_1 + w_2 = 2$ 
 
 $w_1 x_1 +w_2 x_2 = 0$   we obtain $w_1 = \frac{2x_2}{x_2-x_1}$ and $w_2 = \frac{2x_1}{x_1-x_2}$
 
 Using these values for $3^{rd}$ equation $x_1 x_2 =-\frac{1}{3}$ and $4^{th}$ equation $x_1 = -x_2$.


Solving the nonlinear system gives us
<center>
$w_1 =1$, $w_2=1$, $x_1 = - \frac{\sqrt 3}{3}$, $x_2 =  \frac{\sqrt 3}{3}$
</center>
Therefore,

$$\int_{-1}^1 f(x)\,dx \approx f \left( \frac{-\sqrt 3}{3}\right)+ f \left( \frac{\sqrt 3}{3}\right)$$

Above Gauss quadrature integral has degree of precision 3. Trapezoidal rule has degree of precision 1.

|    Number of points, $n$    |                         Weights, $w_i$                                   |                                    Points, $x_i$                                   |
|:------------------------:|:-------------------------------------------------------------------:|:-----------------------------------------------------------------------------------------:|
|    1         |    $2$                                              |    $0$  
|    2         |    $1$                                              |    $\pm \frac{1}{\sqrt 3}$                                              |
|    3         |    $0$                         |    $\frac{8}{9}$                        |
|              |    $\pm \sqrt \frac{3}{5}$          |    $\frac{5}{9}$                        |
|    4         |    $\frac{18+\sqrt{30}}{36}$    |    $\pm \sqrt {\frac{3}{7} -\frac{2}{7} \sqrt{\frac{6}{5}}}$    |
|              |    $\frac{18-\sqrt{30}}{36}$    |    $\pm \sqrt {\frac{3}{7} +\frac{2}{7} \sqrt{\frac{6}{5}}}$    |

# Example

$\int_{-1}^1 \left( x^2+\cos{x} \right)\,dx$ = $\frac{x^3}{3} \bigg\rvert_{-1}^{1} +\sin{x}\bigg\rvert_{-1}^{1} =\frac{2}{3}+2\sin(1)$

Trapezoidal rule :

$\int_{-1}^1 \left( x^2+\cos{x} \right)\,dx \approx \frac{2}{2}\left( f \left( -1\right)+ f \left( 1\right)\right)$

Gauss quadrature:
$\int_{-1}^1 \left( x^2+\cos{x} \right)\,dx \approx f \left( \frac{-\sqrt 3}{3}\right)+ f \left( \frac{\sqrt 3}{3}\right)$


In [22]:
from scipy.integrate import quad
from numpy import sqrt, sin, cos, pi

def f(x):
    return x**2 + cos(x) #integrant

In [23]:
I_actual =2/3 + 2*sin(1) #computed integral

print("Analytical computation of integral       is %1.10f" % I_actual)

#Trapezoidal rule for integral computation
h=2
I_trapezoidal = h*(f(-1)+f(1))/2
error_trapezoidal = abs(I_actual - I_trapezoidal)                   
print("Trapezoidal rule computation of integral is %1.10f and error is %1.10f" % (I_trapezoidal, error_trapezoidal))

Analytical computation of integral       is 2.3496086363
Trapezoidal rule computation of integral is 3.0806046117 and error is 0.7309959755


In [24]:
#Gauss quadrature rule for integral computation
I_Gauss_Quadrature = f(-sqrt(3)/3)+f(sqrt(3)/3)
error_gauss = abs(I_actual - I_Gauss_Quadrature)
print("Analytical computation of integral       is %1.10f" % I_actual)
print("Gauss quadrature computation of integral is %1.10f and error is %1.10f" % (I_Gauss_Quadrature, error_gauss) )

Analytical computation of integral       is 2.3496086363
Gauss quadrature computation of integral is 2.3424903221 and error is 0.0071183142


# Gauss quadrature on arbitrary intervals

Use substition or transformation to transform $\int_{a}^b f(x)\,dx $ into an integral defined over [-1, 1]

Let $x = \frac{1}{2}(a+b)+\frac{1}{2}(b-a)t$, with $t \in [-1, 1]$
Then

$$ \int_{a}^b f(x)\,dx = \int_{-1}^1 f\left(\frac{1}{2}(a+b)+\frac{1}{2}(b-a)t \right) \frac{b-a}{2}\,dt$$


## Summary

* Numerical experiements
* Gauss quadrature concepts:

    * using $n$ quadrature points, a polynomial P(x) of degree (2n – 1) or less will be integrated exactly. 
    * Compared to trapezoidal rule
    
<!-- * When the same number of nodes is used, the algebraic degree of precision of the Gaussian quadrature is higher than that of the Newton-Cotes quadrature. -->